In [1]:
import numpy as np
from collections import defaultdict
import tkinter as tk
import random

In [2]:
# Directions: Up, Right, Down, Left, Wait
ACTIONS = [(-1, 0), (0, 1), (1, 0), (0, -1), (0, 0)]
ACTION_NAMES = ['Up', 'Right', 'Down', 'Left', 'Wait']

In [3]:
class Bot:
    def __init__(self, name, start, goal):
        self.name = name
        self.position = start
        self.goal = goal
        self.path = [start]
        self.movements = []

    def move(self, action):
        self.position = (self.position[0] + action[0], self.position[1] + action[1])
        self.path.append(self.position)
        self.movements.append(ACTION_NAMES[ACTIONS.index(action)])

    def reached_goal(self):
        return self.position == self.goal

In [4]:
class Environment:
    def __init__(self, grid, bots):
        self.original_grid = np.array(grid)
        self.grid = self.original_grid.copy()
        self.bots = bots
        self.height, self.width = self.grid.shape
        self.steps = 0

    def is_valid_move(self, position):
        x, y = position
        return 0 <= x < self.height and 0 <= y < self.width and self.original_grid[x, y] != 'X'

    def get_state(self):
        return tuple(bot.position for bot in self.bots)

    def step(self, actions):
        rewards = np.zeros(len(self.bots))
        new_positions = []

        for i, (bot, action) in enumerate(zip(self.bots, actions)):
            new_pos = (bot.position[0] + ACTIONS[action][0], bot.position[1] + ACTIONS[action][1])
            if self.is_valid_move(new_pos) and new_pos not in new_positions:
                new_positions.append(new_pos)
                bot.move(ACTIONS[action])
                rewards[i] = 100 if bot.reached_goal() else -1
            else:
                new_positions.append(bot.position)
                bot.movements.append('Wait')  # Add 'Wait' if the bot couldn't move
                rewards[i] = -5

        rewards[np.array([new_positions.count(pos) > 1 for pos in new_positions])] -= 10

        self.steps += 1
        self.update_grid()
        done = all(bot.reached_goal() for bot in self.bots)
        return self.get_state(), rewards, done

    def update_grid(self):
        self.grid = self.original_grid.copy()
        for bot in self.bots:
            x, y = bot.position
            self.grid[x, y] = '⛳️' if bot.reached_goal() else bot.name

In [5]:
class QLearningAgent:
    def __init__(self, actions):
        self.q_table = defaultdict(lambda: np.zeros(len(actions)))
        self.epsilon = 0.1
        self.alpha = 0.1
        self.gamma = 0.99

    def get_action(self, state):
        return np.random.randint(len(ACTIONS)) if random.random() < self.epsilon else np.argmax(self.q_table[state])

    def update(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[next_state])
        td_target = reward + self.gamma * self.q_table[next_state][best_next_action]
        td_error = td_target - self.q_table[state][action]
        self.q_table[state][action] += self.alpha * td_error

In [6]:
def train(env, agents, episodes):
    best_episode = None
    best_steps = float('inf')
    best_paths = None
    best_actions = []

    for episode in range(episodes):
        for bot in env.bots:
            bot.position = bot.path[0]
            bot.path = [bot.path[0]]
            bot.movements = []

        env.steps = 0
        state = env.get_state()
        done = False
        episode_actions = []

        while not done and env.steps < best_steps:
            actions = [agent.get_action(state) for agent in agents]
            next_state, rewards, done = env.step(actions)

            episode_actions.append(actions)
            for agent, action, reward in zip(agents, actions, rewards):
                agent.update(state, action, reward, next_state)

            state = next_state

        if done and env.steps < best_steps:
            best_steps = env.steps
            best_episode = episode
            best_paths = [bot.path[:] for bot in env.bots]
            best_actions = episode_actions

    print(f"Training completed. Best episode: {best_episode} with {best_steps} steps")
    return agents, best_paths, best_actions, best_steps

In [7]:
class BotNavigationApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Automated Warehouse Bots")
        self.geometry("600x700")  # Adjusted window size
        self.canvas_size = 500
        self.cell_size = 80
        
        # Create main frame
        self.main_frame = tk.Frame(self)
        self.main_frame.pack(pady=10)

        # Canvas for grid display
        self.canvas = tk.Canvas(self.main_frame, width=self.canvas_size, height=self.canvas_size, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=4)

        # Input fields
        self.rows_entry = self.add_input_field("Number of Rows:", 1)
        self.columns_entry = self.add_input_field("Number of Columns:", 2)
        self.bots_entry = self.add_input_field("Number of Bots:", 3)
        self.obstacles_entry = self.add_input_field("Number of Obstacles:", 4)

        # Buttons
        tk.Button(self.main_frame, text="Set Grid", command=self.setup_grid).grid(row=5, column=0, columnspan=4, pady=10)
        tk.Button(self.main_frame, text="Start Training", command=self.start_training).grid(row=6, column=0, columnspan=4, pady=5)
        tk.Button(self.main_frame, text="Step", command=self.step_through).grid(row=7, column=0, columnspan=4, pady=5)

        self.current_step = 0

    def add_input_field(self, label_text, column):
        frame = tk.Frame(self.main_frame)
        frame.grid(row=column, column=0, sticky="ew", padx=5, pady=5)
        tk.Label(frame, text=label_text).pack(side=tk.LEFT)
        entry = tk.Entry(frame, width=10)
        entry.pack(side=tk.LEFT)
        return entry

    def setup_grid(self):
        rows = int(self.rows_entry.get())
        columns = int(self.columns_entry.get())
        num_bots = int(self.bots_entry.get())
        num_obstacles = int(self.obstacles_entry.get())

    # Initialize grid with empty spaces
        self.grid = np.array([['.'] * columns for _ in range(rows)])

    # Add bots and their goals
        self.bots = []
        for i in range(num_bots):
            bot_row, bot_col = map(int, input(f"Enter Bot {i+1} position (row col): ").split())
            goal_row, goal_col = map(int, input(f"Enter Bot {i+1}'s goal position (row col): ").split())
            bot_name = f"👾{i+1}"
            self.bots.append(Bot(bot_name, (bot_row, bot_col), (goal_row, goal_col)))
        
        # Label the initial position with A1, A2, etc., based on bot name
            self.grid[bot_row, bot_col] = f"A{i+1}"  # Set initial position name
            self.grid[goal_row, goal_col] = f"B{i+1}"  # Set goal position name

        # Print the bot's starting position
            print(f"{bot_name} starts at position: ({bot_row}, {bot_col})")

    # Add obstacles
        for i in range(num_obstacles):
            obs_row, obs_col = map(int, input(f"Enter Obstacle {i+1} position (row col): ").split())
            self.grid[obs_row, obs_col] = 'X'

    # Display the grid
        self.env = Environment(self.grid, self.bots)
        self.draw_grid()


    def draw_grid(self):
        self.canvas.delete("all")
        for i in range(len(self.env.grid)):
            for j in range(len(self.env.grid[0])):
                x1, y1 = j * self.cell_size, i * self.cell_size
                x2, y2 = x1 + self.cell_size, y1 + self.cell_size
                cell_value = self.env.grid[i, j].strip()

                color_fill = {
                    '.': 'white',
                    'X': 'gray',
                }.get(cell_value, 'lightblue')

                for bot in self.env.bots:
                    if bot.position == (i, j):
                        if bot.reached_goal():
                            color_fill = 'light green'
                            cell_value = f"{bot.name} ⛳️"  # Show full bot name and flag
                        else:
                            color_fill = 'lavender'
                            cell_value = bot.name  # Show full bot name
                        break
                    elif bot.goal == (i, j):
                        color_fill = 'light coral'
                        for b in self.env.bots:
                            if b.position == (i, j):
                                cell_value = f"{b.name}"  # Show bot name when passing through other bot's destination
                                break
                            else:
                                cell_value = f"B{bot.name[1:]}"  # Show destination name
                        break
                    elif cell_value.startswith("A"):  # Check if cell is initial position
                        color_fill = 'lightblue'
                        for b in self.env.bots:
                            if b.position == (i, j):
                                cell_value = f"A{b.name[1:]}"  # Keep initial position name
                                break

                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color_fill, outline="black")
                if cell_value != '.':
                    self.canvas.create_text((x1 + x2) / 2, (y1 + y2) / 2, text=cell_value, fill='black', font=('Helvetica', 10))  # Adjust font size to fit the text
    def start_training(self):
        global trained_agents, best_paths, best_actions, best_steps
        trained_agents, best_paths, best_actions, best_steps = train(self.env, [QLearningAgent(ACTIONS) for _ in range(len(self.bots))], episodes=50000)
        print(f"Best Episode Navigation (steps: {best_steps}):")
        for bot in self.env.bots:
            bot.position = bot.path[0]
            bot.path = [bot.path[0]]
            bot.movements = []
        self.env.steps = 0
        self.env.update_grid()
        self.draw_grid()

    def step_through(self):
        if self.current_step < len(best_actions):
            actions = best_actions[self.current_step]
            state, rewards, done = self.env.step(actions)
            print(f"Step {self.current_step + 1}:")
            print(self.env.grid)
            self.current_step += 1
            self.env.update_grid()
            self.draw_grid()
            self.update()
            if done:
                print(f"All bots have reached their destinations in {self.env.steps} steps!")
                tk.Label(self, text=f"All bots reached the goal in {self.env.steps} steps!").pack()
                self.print_bot_movements()
        else:
            print("Reached end of steps.")

    def print_bot_movements(self):
        for bot in self.env.bots:
            print(f"{bot.name} movements: {' -> '.join(bot.movements)}")

if __name__ == "__main__":
    app = BotNavigationApp()
    app.mainloop()